```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Best Practices for CI in Python}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

## What is CI?

Before diving into the best practices of Continuous integration,
it's a good idea to understand what it is.
Specifically,
breaking down CI into its component parts.

```{=latex}
\begin{frame}
\frametitle{What is Continuous Integration made of?}

The gears in the machine

\end{frame}
```

Before having CI is even expected,
we usually expect that it is possible to run a
local build.
This is not strictly a requirement,
but it a highly desirable property.
If a build fails,
not being able to simulate locally is unpleasant.

As a working assumption,
there is already a way to run,
locally:

* Lint
* Test
* Package

```{=latex}
\begin{frame}
\frametitle{(Usual) Requirements for Continuous Integration}

Local "build":\pause

Lint\pause

Test\pause

Package

\end{frame}
```

Assuming these steps can be at each point of the source code,
when does the server run them,
and on which source code versions?

In the old days,
it was the
"nightly build".
This might seem ridiculous to modern ears,
but at some point,
having a nightly build at all was considered
advanced.


The next step up is automatically building on a merge
to the main development branch.
This means that a broken main branch is immediately detected,
and,
in general,
can be attributed to one change.

The modern way is to run the build on a
*suggested*
patch.
Some systems call it a
"try"
run,
some call it running the CI
on a
pull request
or a merge request.

In all of those cases,
the advantage is that there is feedback on a proposed change
*before*
it goes it.
Often,
patches with a failing CI
cannot be
applied
(or "merged"),
or sometimes need special approval to merge.


There is other exotica:
running builds on
"merge trains"
and then only running the build on intermediate steps
if the build fails,
running the build against a fast-forwarded version of the
patch or similar.


```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Run Build on Server}

When? \pause

Classic: Nightly \pause

Modern: Merge to main \pause

Advanced: Suggested patch (Pull Request, Merge Request) \pause

Exotica (Merge trains and more)

\end{frame}
```

The focus of the talk will be on
"patch builds"
(or
"PR builds"
or
"MR builds"
or
"RR builds").
In those cases,
the main use of the CI
is as an automated gate:
"is the patch good?"

Often,
the CI being
"green"
will be a prerequisite for a human reviewing it.
As above,
in some cases,
a reviewer can override the CI
and approve a
"red"
patch.

This should be a rare occasion.
If red approval happens too often,
the CI is not delivering enough value.



```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Patch Builds}

Focus of talk\pause

Automated gate: "is patch good?"

\end{frame}
```

The continuous integration is a form of computation.
The input is a source code snapshot,
and the output is the CI results.

The thing that runs the computation is called a
"runner".
Another popular term is a
"worker".

Usually,
a centeralized
or semi-centralized
"coordinator"
will send jobs to the workers and runners.
It will collect,
and backup,
the results.

```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Runners}

Architecture:

\begin{itemize}
\item CI coordinator
\item CI runners
\end{itemize}

\end{frame}
```

Most modern frameworks collect live logs,
to allow for real-time feedback.
Post run,
they retain the log
"forever",
possibly with a configurable retention policy.

```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Build logs}

Modern frameworks:\pause

Collect live from runners\pause

Retain "forever"

\end{frame}
```

## What is Good CI?


Given that this is what a CI system
*is*,
what is a good CI?
Especially in the context of Python.

This does not help achieve it,
but it does paint the goal to achieve.

```{=latex}
\begin{frame}
\frametitle{What makes CI good?}

Paint the target

\end{frame}
```

Ideally,
a CI is
*accurate*.
When it says a patch is good,
it is good.
When it says a patch is bad,
it is bad.


```{=latex}
\begin{frame}
\frametitle{CI criteria: accuracy}

Is the answer correct?

\end{frame}
```

When a patch fails,
good feedback
is
*actionable*.
What was the problem?
How can it be fixed?
How can it be reproduced locally?

```{=latex}
\begin{frame}
\frametitle{CI criteria: actionability}

If patch is not good, how clear is it how to fix? \pause

How to reproduce locally?

\end{frame}
```

Whether good or bad,
fast feedback is good feedback.
The situation is not as symmetric as that,
though.

Red feedback promptness is not important.
Green feedback can be semi-automated:
the next step is to ask for a review,
or,
if pre-reviewed,
can be automerged on success.

Red feedback, however,
needs to be fixed.
This is part of the developer's loop,
and should be optimized with care.

```{=latex}
\begin{frame}
\frametitle{CI criteria: promptness}

How long does it take to answer?

\end{frame}
```

Finally,
cost.
Cost is important to consider because many of the other criteria can be improved using costs.

Actionable feedback takes more resources.
Running tests more than once makes them more accurate.
Run tests in parallel, and on stronger machines,
makes them faster.


```{=latex}
\begin{frame}
\frametitle{CI criteria: cost}

Mostly the runner compute cost

\end{frame}
```

## Improving accuracy

How do you improve the accuracy of the CI pipeline?
One of the main sources in inaccuracies,
both false alarms and missing alarms,
are different environments.

When at all possible,
run tests inside containers.
Pinned container images that are custom-built.
They do not need to be
*built from scratch*,
but they need to be custom-built and pinned.

Include the version of Python you want,
and any non-Python-library dependencies,
in the container.

```{=latex}
\begin{frame}
\frametitle{CI accuracy: use containers}

Container images with: \pause

Version of Python\pause

Other non-Python dependencies \pause

Pin the image tag!

\end{frame}
```

Another source of inaccuracy
is different versions of
PyPI libraries.
Many programs have hundreds or even thousands
of recursive dependencies.

All CI runs,
including tests,
linting,
docs,
type checking,
and more,
should use pinned dependencies
(`requirements.txt`,
or alternatives when using other
virtual environment managers).

You should upgrade the pins --
in a dedicated patch,
that does nothing else.
There are services to do that,
but doing it manually is better
than
YOLOing it.


```{=latex}
\begin{frame}
\frametitle{CI accuracy: pin versions}

Test against pinned dependencies\pause

Upgrade pins in a dedicated patch\pause

(There are services)

\end{frame}
```

Finally,
monitor and improve your test quality.
Tests should be written to detect real issues,
and avoid detecting spurious problems.

*Improving*
unit tests is its own,
ten hour,
talks.
From the perspective of a CI
system,
the most important thing is
*monitoring*.

Can you tell which tests are flakey?
Can you tell how many bugs pass the test suites?
How are you tracking that?

```{=latex}
\begin{frame}
\frametitle{CI accuracy: test quality}

Monitor and improve test quality\pause

(This is a whole 'nother talk)

\end{frame}
```

## Improving actionability

Many test/lint runners
keep verbosity,
by default,
to a happy medium.
In CI,
the trade-offs are different.

Set verbosity to the maximum it will allow.
If this makes logs completely unreadable,
use tricks like
`tee | filter`
and allow downloading the raw logs.
Failures in CI,
especially those that do not reproduce locally,
are horrendous to debug.

Anything that makes them easier is worthwhile.

```{=latex}
\begin{frame}
\frametitle{CI actionability: set verbosity to 11}

Use verbosity options in test runners/linters/etc.\pause

Logs can be filtered more easily than unfiltered

\end{frame}
```

When tests fail,
they should fail with relevant details.
Sure,
`False is not True`,
but you already knew that.
What parameters made the False?
Which one of them was constant?

Some frameworks,
like
`pytest`,
will help:
but they are not magic.
When writing tests,
force them to fail,
and check that they are verbose.

Exceptions you raise yourself get no special treatment.
Make sure that they contain enough information so that
when they appear in the logs,
they have information to help diagnose the problem.

You are also probably better off setting logging levels
to high verbosity,
and directing them to a file that can be downloaded.
This can be invaluable in tracking down issues.

```{=latex}
\begin{frame}
\frametitle{CI actionability: test failure verbosity}

Test assertion failure in test for verbosity\pause

When raising exception, add details!


\end{frame}
```

Before starting the test,
help understand the context by
spewing your guts to the log.
Talk about the enviornment variables,
the platform,
and anything else that might be useful.

As a useful corrollary,
avoid putting secrets in environment variables,
especially persistent secrets.

```{=latex}
\begin{frame}
\frametitle{CI actionability: environmental details}

Spew details on environment to logs\pause

Environment variables\pause

Platform\pause

Etc.\pause

(Don't put secrets in environment)


\end{frame}
```

## Improving promptness

There is nothing more frustrating than waiting for a computer.
Time waste is death my a thousand cuts.

Any primitives the CI system itself has to cache downloads locally
should be used
(or at least tested).
When building containers,
use container layer caching.
This can be non-trivial to set up,
in order to cache against a registry.
It is worth it.

Cache downloads from PyPI locally.
"Locally" is,
as always,
a matter of degree.
If it helps, it helps.
No need to litigate what
"here"
means
like you're on an episode of Sesame street.

```{=latex}
\begin{frame}
\frametitle{CI promptness: caching}

Use CI primitives to cache downloads\pause

Container layer caching\pause (complicated but worth it)\pause

Local PyPI and Container caching proxies

\end{frame}
```

Use pooling primitives to reuse connections and downloads
for anything that needs to connect remotely.
While in general unit tests should avoid connectivity,
sometimes this is unavoidable.
In those cases,
inter-test pooling can help quite a bit.



```{=latex}
\begin{frame}
\frametitle{CI promptness: pooling}

Reuse connections, downloads, etc.\pause

Think carefully how to break up tests

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI promptness: fail fast}

Order tests based on likelihood to fail\pause

Open source solutions exist

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI promptness: parallelize}

Use CI primitives to parallelize independent runs

\end{frame}
```

## Improving cost

```{=latex}
\begin{frame}
\frametitle{CI cost: kill useless runs}

Example: Commit added to patch

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI cost: stop runs early}

Do you need to finish if it fails? \pause

(Sometimes! Trade-offs)

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI cost: better tests}

Examples: Better stubbing/mocking instead of real services
\end{frame}
```

## Summary

```{=latex}
\begin{frame}
\frametitle{CI quality: trade-offs}

Effort \pause

Quality \pause

Cost \pause

Decide!


\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI quality: measure}

Given trade-offs, is this ok?


\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI quality: improve}

What needs to be better?

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI quality: repeat}

Constant vigilance!

\end{frame}
```

```{=latex}
\end{document}
```